# eBOSS

### Imports

In [ ]:
# some_file.py
import sys
import yaml
# insert at 1, 0 is the script path (or '' in REPL)

sys.path.insert(1, '/home/jaimerz/PhD/xCell')

from xcell.mappers import MappereBOSS
from xcell.mappers import utils
from xcell.cls.data import Data
from xcell.cls.cl import Cl
from xcell.cls.cov import Cov
from astropy.io import fits
from astropy.table import Table
import pyccl as ccl
import numpy as np
import pylab as plt
import pymaster as nmt
import healpy as hp
import numpy as np

### Configurations

In [ ]:
data = Data(data_path="../config_files/master.yml")

### Mappers

In [ ]:
QSO_0 = data.get_mapper("SDSS__QSO0")
QSO_1 = data.get_mapper("SDSS__QSO1")
LRG = data.get_mapper("SDSS__LRG")
ELG = data.get_mapper("SDSS__ELG")

In [ ]:
QSO_0_map = QSO_0.get_signal_map()
QSO_1_map = QSO_1.get_signal_map()
LRG_map = LRG.get_signal_map()
ELG_map = ELG.get_signal_map()

In [ ]:
hp.visufunc.mollview(map=np.array(QSO_0_map)[0])

In [ ]:
hp.visufunc.mollview(map=np.array(QSO_1_map)[0])

In [ ]:
hp.visufunc.mollview(map=np.array(LRG_map)[0])

In [ ]:
hp.visufunc.mollview(map=np.array(ELG_map)[0])

### Cl's

In [ ]:
cl_QSO0_QSO0 = Cl(data.data, "SDSS__QSO0", "SDSS__QSO0")
cl_QSO1_QSO1 = Cl(data.data, "SDSS__QSO1", "SDSS__QSO1")
cl_LRG_LRG = Cl(data.data, "SDSS__LRG", "SDSS__LRG")
cl_ELG_ELG = Cl(data.data, "SDSS__ELG", "SDSS__ELG")

cl_QSO0_QSO0 = cl_QSO0_QSO0.get_ell_cl()
cl_QSO1_QSO1 = cl_QSO1_QSO1.get_ell_cl()
cl_LRG_LRG = cl_LRG_LRG.get_ell_cl()
cl_ELG_ELG = cl_ELG_ELG.get_ell_cl()

### Covariance Matrix

In [ ]:
cov_QSO0_QSO0_QSO0_QSO0 = Cov(data.data, "SDSS__QSO0", "SDSS__QSO0", "SDSS__QSO0", "SDSS__QSO0")
cov_QSO1_QSO1_QSO1_QSO1 = Cov(data.data, "SDSS__QSO1", "SDSS__QSO1", "SDSS__QSO1", "SDSS__QSO1")
cov_LRG_LRG_LRG_LRG = Cov(data.data, "SDSS__LRG", "SDSS__LRG", "SDSS__LRG", "SDSS__LRG")
cov_ELG_ELG_ELG_ELG = Cov(data.data, "SDSS__ELG", "SDSS__ELG", "SDSS__ELG", "SDSS__ELG")

cov_QSO0_QSO0_QSO0_QSO0 = cov_QSO0_QSO0_QSO0_QSO0.get_covariance()
cov_QSO1_QSO1_QSO1_QSO1 = cov_QSO1_QSO1_QSO1_QSO1.get_covariance()
cov_LRG_LRG_LRG_LRG = cov_LRG_LRG_LRG_LRG.get_covariance()
cov_ELG_ELG_ELG_ELG = cov_ELG_ELG_ELG_ELG.get_covariance()

In [ ]:
err_QSO0_QSO0 = np.sqrt(np.diag(cov_QSO0_QSO0_QSO0_QSO0))
err_QSO1_QSO1 = np.sqrt(np.diag(cov_QSO1_QSO1_QSO1_QSO1))
err_LRG_LRG = np.sqrt(np.diag(cov_LRG_LRG_LRG_LRG))
err_ELG_ELG = np.sqrt(np.diag(cov_ELG_ELG_ELG_ELG))

### Theory Cl's

In [ ]:
cosmo = ccl.Cosmology(Omega_c=0.27, Omega_b=0.049, h=0.67, sigma8=0.81, n_s=0.96)

QSO_0_nz = QSO_0.get_nz()
QSO_1_nz = QSO_1.get_nz()
LRG_nz = LRG.get_nz()
ELG_nz = ELG.get_nz()

QSO_0_z = QSO_0_nz[0]
QSO_0_dNdz = QSO_0_nz[1]
QSO_1_z = QSO_1_nz[0]
QSO_1_dNdz = QSO_1_nz[1]
LRG_z = LRG_nz[0]
LRG_dNdz = LRG_nz[1]
ELG_z = ELG_nz[0]
ELG_dNdz = ELG_nz[1]

b_QSO_0 = data.data["tracers"]["SDSS__QSO0"]["bias"]*np.ones(len(QSO_0_z))
b_QSO_1 = data.data["tracers"]["SDSS__QSO1"]["bias"]*np.ones(len(QSO_1_z))
b_LRG = data.data["tracers"]["SDSS__LRG"]["bias"]*np.ones(len(LRG_z))
b_ELG = data.data["tracers"]["SDSS__ELG"]["bias"]*np.ones(len(ELG_z))

tracer_QSO0 = ccl.NumberCountsTracer(cosmo, has_rsd=False, dndz=(QSO_0_z, QSO_0_dNdz), bias=(QSO_0_z, b_QSO_0))
tracer_QSO1 = ccl.NumberCountsTracer(cosmo, has_rsd=False, dndz=(QSO_1_z, QSO_1_dNdz), bias=(QSO_1_z, b_QSO_1))
tracer_LRG = ccl.NumberCountsTracer(cosmo, has_rsd=False, dndz=(LRG_z, LRG_dNdz), bias=(LRG_z, b_LRG))
tracer_ELG = ccl.NumberCountsTracer(cosmo, has_rsd=False, dndz=(ELG_z, ELG_dNdz), bias=(ELG_z, b_ELG))

clt_QSO0_QSO0 = ccl.angular_cl(cosmo, tracer_QSO0, tracer_QSO0, cl_QSO0_QSO0[0]) 
clt_QSO1_QSO1 = ccl.angular_cl(cosmo, tracer_QSO1, tracer_QSO1, cl_QSO1_QSO1[0]) 
clt_LRG_LRG = ccl.angular_cl(cosmo, tracer_LRG, tracer_LRG, cl_LRG_LRG[0])  
clt_ELG_ELG = ccl.angular_cl(cosmo, tracer_ELG, tracer_ELG, cl_ELG_ELG[0]) 

### Comparison

In [ ]:
plt.plot(ELG_z, ELG_dNdz, label='ELG')
plt.plot(LRG_z, LRG_dNdz, label='LRG')
plt.legend()
plt.show()

In [ ]:
plt.plot(QSO_0_z, QSO_0_dNdz, label='QSO_0')
plt.plot(QSO_1_z, QSO_1_dNdz, label='QSO_1')
plt.legend()
plt.show()

In [ ]:
plt.errorbar(cl_QSO0_QSO0[0], cl_QSO0_QSO0[1][0], yerr=err_QSO0_QSO0, fmt='o-', label='data')
plt.plot(cl_QSO0_QSO0[0], clt_QSO0_QSO0, 'o-', label='theory')
plt.title('QSO0-QSO0')
plt.xlabel('$\\ell$', fontsize=16)
plt.ylabel('$C_\\ell$', fontsize=16)
plt.legend(loc='lower left', ncol=2, labelspacing=0.1)
plt.xscale("log")
plt.yscale("log")
plt.show()

In [ ]:
plt.errorbar(cl_QSO1_QSO1[0], cl_QSO1_QSO1[1][0], yerr=err_QSO1_QSO1, fmt='o-', label='data')
plt.plot(cl_QSO1_QSO1[0], clt_QSO1_QSO1, 'o-', label='theory')
plt.title('QSO1-QSO1')
plt.xlabel('$\\ell$', fontsize=16)
plt.ylabel('$C_\\ell$', fontsize=16)
plt.legend(loc='lower left', ncol=2, labelspacing=0.1)
plt.xscale("log")
plt.yscale("log")
plt.show()

In [ ]:
plt.errorbar(cl_LRG_LRG[0], cl_LRG_LRG[1][0], yerr=err_LRG_LRG, fmt='o-', label='data')
plt.plot(cl_LRG_LRG[0], clt_LRG_LRG, 'o-', label='theory')
plt.title('LRG-LRG')
plt.xlabel('$\\ell$', fontsize=16)
plt.ylabel('$C_\\ell$', fontsize=16)
plt.legend(loc='lower left', ncol=2, labelspacing=0.1)
plt.xscale("log")
plt.yscale("log")
plt.show()

In [ ]:
plt.errorbar(cl_ELG_ELG[0], cl_ELG_ELG[1][0], yerr=err_ELG_ELG, fmt='o-', label='data')
plt.plot(cl_ELG_ELG[0], clt_ELG_ELG, 'o-', label='theory')
plt.title('ELG-ELG')
plt.xlabel('$\\ell$', fontsize=16)
plt.ylabel('$C_\\ell$', fontsize=16)
plt.legend(loc='lower left', ncol=2, labelspacing=0.1)
plt.xscale("log")
plt.yscale("log")
plt.show()

In [ ]:
SDSS_QSO0 = {
    "nz": QSO0_nz,
    "cl": cl_QSO0_QSO0,
    "err": err_QSO0_QSO0,
    }
SDSS_QSO1 = {
    "nz": QSO1_nz,
    "cl": cl_QSO1_QSO1,
    "err": err_QSO1_QSO1,
    }
SDSS_LRG = {
    "nz": LRG_nz,
    "cl": cl_LRG_LRG,
    "err": err_LRG_LRG,
    }
SDSS_ELG = {
    "nz": ELG_nz,
    "cl": cl_ELG_ELG,
    "err": err_ELG_ELG,
    }

In [ ]:
np.savez("../results/SDSS_QSO0", SDSS_QSO0)
np.savez("../results/SDSS_QSO1", SDSS_QSO1)
np.savez("../results/SDSS_LRG", SDSS_LRG)
np.savez("../results/SDSS_ELG", SDSS_ELG)

In [ ]:
import shutil
shutil.rmtree("../reruns/")